# Make test data for NiMARE
This reduces the 21 pain study NiMARE dataset to 10x10x10 maps. Plus adds an atlas and a mask.

In [7]:
import nimare
from nimare.extract import download_nidm_pain
from nimare.tests.utils import get_test_data_path
import os.path as op
import nibabel as nib
import numpy as np

In [ ]:
nidm_path = download_nidm_pain()
dset_file = op.join(get_test_data_path(), 'nidm_pain_dset.json')
out_dir = op.join(get_test_data_path(), 'test_pain_dataset')

In [ ]:
dset = nimare.dataset.Dataset(dset_file)
dset.update_path(nidm_path)
for target in ['varcope', 'z']:
    dset.images = nimare.transforms.transform_images(
        dset.images, target=target, metadata_df=dset.metadata, 
        masker=dset.masker)

In [ ]:
for id_ in dset.ids:
    for imgtype in dset.get_images(ids=id_):
        in_file = dset.get_images(ids=id_, imtype=imgtype)
        img = nib.load(in_file)
        data = img.get_fdata()[40:50, 45:55, 40:50]
        # We will drop some voxels for the first five studies,
        # to test missing data performance
        if id_ in dset.ids[:5]:
            data[:3, :3, :3] = 0
        new_img = nib.Nifti1Image(data, img.affine, header=img.header)
        out_file = op.join(out_dir, '{}_{}.nii.gz'.format(id_.replace('.nidm-1', ''), imgtype))
        new_img.to_filename(out_file)

In [9]:
# make mask
img = nib.load(op.join(out_dir, 'pain_01_z.nii.gz'))
data = img.get_fdata()
mask_data = np.ones(data.shape, int)
mask_img = nib.Nifti1Image(mask_data, img.affine, header=img.header)
mask_img.to_filename(op.join(out_dir, 'mask.nii.gz'))

In [11]:
# make atlas for tests
atlas_data = np.zeros(data.shape, int)
atlas_data[:2, :2, :2] = 1
atlas_data[2:4, :2, :2] = 2
atlas_data[4:6, :2, :2] = 3
atlas_data[6:8, :2, :2] = 4
atlas_data[8:10, :2, :2] = 5
atlas_img = nib.Nifti1Image(atlas_data, img.affine, header=img.header)
atlas_img.to_filename(op.join(out_dir, 'atlas.nii.gz'))